jetzt sim

In [2]:
from src.sampling import sample_salpeter, sample_metallicity
from src.stellar_physics import ms_lifetime_years, stellar_endpoint
from src.supernovae import is_type_ia, core_collapse_SN_type

Monte-Carlo-Simulation

population ziehen und sicherstellen

In [3]:
N = 100000
import numpy as np  
masses = sample_salpeter(N)
masses = np.atleast_1d(masses)
metallicities = sample_metallicity(N)
metallicities = np.atleast_1d(metallicities)
print("Sampled masses:", masses)
print("Sampled metallicities:", metallicities)

Sampled masses: [0.32228267 0.11449912 0.17108711 ... 0.28271422 0.17377308 0.12109559]
Sampled metallicities: [0.0291897  0.02464871 0.0245576  ... 0.02488262 0.00681037 0.00187977]


einzelstern physik anwenden :P

In [6]:
lifetimes = np.array([
    ms_lifetime_years(masses, metallicities)
    for masses, metallicities in zip(masses, metallicities)
])
print("Computed lifetimes:", lifetimes)

endstates = np.array([
    stellar_endpoint(m) 
    for m in np.array(masses)
    ])
print("Determined stellar endpoints:", endstates)

Computed lifetimes: [1.96439100e+11 2.52422381e+12 9.24205135e+11 ... 2.63988183e+11
 6.87784337e+11 1.31151868e+12]
Determined stellar endpoints: ['WD' 'WD' 'WD' ... 'WD' 'WD' 'WD']


supernovaeeee

In [8]:
cc_sn_types = np.array([
    core_collapse_SN_type(m)
    for m in masses
    ])  
print("Determined core-collapse SN types:", cc_sn_types)

ia_flags = np.array([
    is_type_ia(m)
    for m in masses
    ])  
print("Determined Type Ia flags:", ia_flags)

sn_types = np.where(ia_flags, "Type Ia", cc_sn_types)
print("Determined supernova types:", sn_types)

Determined core-collapse SN types: ['No SN' 'No SN' 'No SN' ... 'No SN' 'No SN' 'No SN']
Determined Type Ia flags: [False False False ... False False False]
Determined supernova types: ['No SN' 'No SN' 'No SN' ... 'No SN' 'No SN' 'No SN']
